In [1]:
import glob
import os
import sys
from queue import Queue
from queue import Empty
import random
import time
import numpy as np
import cv2
from tensorflow.keras.models import load_model
from PIL import Image


try:
    sys.path.append(
        glob.glob("../carla/dist/carla-*%d.%d-%s.egg"
            % (sys.version_info.major,sys.version_info.minor,"win-amd64" if os.name == "nt" else "linux-x86_64",))[0])
except IndexError:
    pass

import carla

CAMERA_POSITION =  [1.3, 0.0, 2.3]
CAMERA_HEIGHT = 480
CAMERA_WIDTH = 960
CAMERA_FOV = 120
CAMERA_LEFT = -60.0
CAMERA_RIGHT = 60.0

# IMAGE
IMAGE_WIDTH = 320
IMAGE_RESOLUTION = (160, 704)

COMBINED_IMAGE_SHAPE = (160, 704, 3)

MODEL_INPUT_SIZE = (10,) + COMBINED_IMAGE_SHAPE

MODEL_OUTPUT_SIZE = (5,)


In [2]:
def process_img(image, scale=1, start_x=0, crop_x=None, start_y=0, crop_y=None):
    i = np.array(image.raw_data)
    i2 = i.reshape((image.height, image.width, 4))
    i3 = i2[:, :, :3]
    """ cv2.imshow("", i3)
    cv2.waitKey(1) """
    #i3/255.0
    i3 = Image.fromarray(i3)
    (width, height) = (i3.width // scale, i3.height // scale)
    if crop_x is None:
        crop_x = width
    if crop_y is None:
        crop_y = height
        
    i4 = np.asarray(i3)
    cropped_image = i4[start_y:start_y+crop_y, start_x:start_x+crop_x]
    return cropped_image#/255.0

def sensor_callback(sensor_data, sensor_queue, sensor_name):
    processed_img = process_img(sensor_data, 1, IMAGE_WIDTH, IMAGE_WIDTH, IMAGE_RESOLUTION[0], IMAGE_RESOLUTION[0])
    sensor_queue.put((processed_img, sensor_name))

In [3]:

sensor_list = []
def camera_rgb_install():
    camera_positions = [
        carla.Transform(
            carla.Location(x=CAMERA_POSITION[0], z=CAMERA_POSITION[2]),
            carla.Rotation(yaw=CAMERA_LEFT),
        ),  # Left
        carla.Transform(
            carla.Location(x=CAMERA_POSITION[0], z=CAMERA_POSITION[2])
        ),  # Center
        carla.Transform(
            carla.Location(x=CAMERA_POSITION[0], z=CAMERA_POSITION[2]),
            carla.Rotation(yaw=CAMERA_RIGHT),
        ),
    ]  # Right

    cameras = []
    for camera_transform in camera_positions:
        camera_blueprint = world.get_blueprint_library().find("sensor.camera.rgb")
        camera_blueprint.set_attribute("image_size_x", str(CAMERA_WIDTH))
        camera_blueprint.set_attribute("image_size_y", str(CAMERA_HEIGHT))
        camera_blueprint.set_attribute("fov", str(CAMERA_FOV))
        #camera_blueprint.set_attribute("sensor_tick", "0.1")
        camera = world.spawn_actor(
            camera_blueprint, camera_transform, attach_to=ego_vehicle
        )
        actor_list.append(camera)
        cameras.append(camera)
        sensor_list.append(camera)
    return cameras


actor_list = []

client = carla.Client("localhost", 2000)
client.set_timeout(3.0)

# Once we have a client we can retrieve the world that is currently running.
world = client.get_world()

try:
    # We need to save the settings to be able to recover them at the end
    # of the script to leave the server in the same state that we found it.
    original_settings = world.get_settings()
    settings = world.get_settings()

    # We set CARLA syncronous mode
    settings.fixed_delta_seconds = 0.05  # Fixed time-step == 10 FPS (1 / 0,1)
    settings.synchronous_mode = True
    world.apply_settings(settings)

    sensor_queue = Queue()

    # Retrieve the spectator object
    spectator = world.get_spectator()

    # Get the location and rotation of the spectator through its transform
    transform = spectator.get_transform()

    location = transform.location
    rotation = transform.rotation

    blueprint_library = world.get_blueprint_library()

    # Set the spectator with an empty transform
    # spectator.set_transform(carla.Transform())

    # This will set the spectator at the origin of the map, with 0 degrees
    # pitch, yaw and roll - a good way to orient yourself in the map

    vehicle_blueprints = blueprint_library.filter("*vehicle*")

    spawn_points = world.get_map().get_spawn_points()

    ego_vehicle = world.spawn_actor(
        blueprint_library.filter("etron")[0], random.choice(spawn_points)
    )
    actor_list.append(ego_vehicle)
    ego_transform = ego_vehicle.get_transform()

    spectator_location = ego_transform.location + carla.Location(0, 0, 100)
    spectator_rotation = ego_transform.rotation
    spectator.set_transform(
        carla.Transform(
            spectator_location,
            spectator_rotation,
        )
    )
    spectator_rotation.pitch = -90

    """ for i in range(0, 50):
        npc_vehicle = world.try_spawn_actor(
            random.choice(vehicle_blueprints), random.choice(spawn_points)
        )
        if npc_vehicle is not None:
            # npc_vehicle.set_autopilot(True)
            actor_list.append(npc_vehicle) """

    """ postavljanje senzora na auto """
    # kamera RGB
    camera_list = camera_rgb_install()

    ego_vehicle.get_control().steer = 0.0
    ego_vehicle.get_control().throttle = 0.0
    ego_vehicle.get_control().brake = 0.0
    ego_vehicle.get_control().hand_brake = False
    ego_vehicle.get_control().reverse = False

    """ while True:
        world.tick()

        ego_vehicle.get_control().throttle = 0.3


        shape = (3, IMAGE_WIDTH, IMAGE_RESOLUTION[0], 3)
        image_queue = np.empty((10, 160, 704, 3), dtype=np.float32)

        camera_images_np = np.empty(shape, dtype=np.float32)
        camera_images = []

        for camera in camera_list:
            camera.listen(lambda data: sensor_callback(data, sensor_queue, "camera%d" % camera_list.index(camera)))

        try:
            for _ in range(len(sensor_list)):
                s_frame = sensor_queue.get(True, 1.0)
                print("    Frame: %d   Sensor: %s" % (s_frame[0], s_frame[1]))

        except Empty:
            print("    Some of the sensor information is missed") """






    seconds = 3
    image_queue = np.empty((10, 160, 704, 3), dtype=np.float32)
    for sec in range(seconds):

        # Synchronize the CARLA world
        world.tick()

        ego_vehicle.get_control().throttle = 0.3


        shape = (3, IMAGE_WIDTH, IMAGE_RESOLUTION[0], 3)

        camera_images_np = np.empty(shape, dtype=np.float32)
        camera_images = []

        camera_images_np = np.empty(shape, dtype=np.float32)
        combined_image = np.empty(COMBINED_IMAGE_SHAPE, dtype=np.float32)
        camera_images = []

        for camera in camera_list:
            camera.listen(lambda data: sensor_callback(data, sensor_queue, "camera%d" % camera_list.index(camera)))

        while len(sensor_list) != 3:
            pass
        try:
            for _ in range(len(sensor_list)):
                s_frame = sensor_queue.get(True, 1.0)
                print("    Frame: %d   Sensor: %s" % (s_frame[0], s_frame[1]))

        except Empty:
            print("    Some of the sensor information is missed") 

        """ for camera in camera_list:
            camera.listen(lambda image: camera_images.append(process_img(image, 1, IMAGE_WIDTH, IMAGE_WIDTH, IMAGE_RESOLUTION[0], IMAGE_RESOLUTION[0])))
        
        time.sleep(0.1)

        print("Camera Images before: ", len(camera_images))
        time.sleep(0.1)
        print("Camera Images after: ", len(camera_images))
        while len(camera_images) < len(camera_list):
            continue

        combined_image = np.concatenate(np.array(camera_images), axis=1)
        cv2.imwrite("images/combined_image.png", combined_image)

        np.append(image_queue,combined_image)
        print("shape: ", image_queue.shape)
        cv2.imwrite("images/image_queue.png", image_queue[0]) """

        if (len(image_queue) == 10):
            print("model predict")

       
   

finally:
    world.apply_settings(original_settings)
    for actor in actor_list:
        actor.destroy()
    print("done.")

: 

: 